<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Battery_raw_dataclean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data cleaning and normalization of raw data

In this demo, the raw data extracted by chemdataextractor_batteries will be cleaned and normalized including the following steps: removing the incorrect data records, processing compound names, and creating different versions of databases. 

In [1]:
import json
import pandas as pd
raw_data = []
fileroot = 'raw_data.json'
with open(fileroot, encoding='utf-8') as f:
    for line in f:
        raw_data.append(json.loads(line))

In [2]:
raw_data

[{'BatteryVoltage': {'compound': {'Compound': {'names': ['LiFePO4']}},
   'names': 'LiFePO4',
   'raw_units': 'V',
   'raw_value': '3.7',
   'specifier': 'voltage',
   'units': 'Volt^(1.0)',
   'value': 3.7},
  'metadata': {'authors': ['Alex H.\xa0Johnstone', 'Alex H.\xa0Johnstone'],
   'date': '2000/01/01',
   'doi': '10.1039/A9RP90001B',
   'firstpage': '9',
   'html_url': 'https://pubs.rsc.org/en/content/articlelanding/2000/rp/a9rp90001b',
   'issue': '1',
   'journal': 'Chemistry Education Research and Practice',
   'language': 'en',
   'lastpage': '15',
   'pdf_url': 'https://pubs.rsc.org/en/content/articlepdf/2000/rp/a9rp90001b',
   'publisher': 'Royal Society of Chemistry',
   'title': 'TEACHING OF CHEMISTRY - LOGICAL OR PSYCHOLOGICAL? ',
   'volume': '1'},
  'warning': 1},
 {'BatteryCapacity': {'compound': {'Compound': {'names': ['graphite']}},
   'names': 'graphite',
   'raw_units': 'mAh/g',
   'raw_value': '372',
   'specifier': 'capacity',
   'units': 'Gram^(-1.0)  Hour^(1.0

In [3]:
def convert_rawdata(raw_data):
    """
    Data tables created with columns including Property, Name, Raw_value, Raw_unit, Value, Unit,
    DOI, Title, Journal, Date, Specifier, Correctness, Tag, Info, and Warning.
    """
    new_dic = {}
    property_raw = list(raw_data.keys())[0]
    propertyname = property_raw.split('Battery')[-1]
    new_dic['Property'] = propertyname
    new_dic['name'] = raw_data[property_raw]['compound']['Compound']['names']
    new_dic['Raw_value'] = raw_data[property_raw]['raw_value']
    new_dic['Raw_unit'] = raw_data[property_raw]['raw_units']
    new_dic['Value'] = raw_data[property_raw]['value']
    new_dic['Unit'] = raw_data[property_raw]['units']
    
    new_dic['DOI'] = raw_data['metadata']['doi']
    new_dic['Title'] = raw_data['metadata']['title']
    new_dic['Journal'] = raw_data['metadata']['journal']
    new_dic['Date'] = raw_data['metadata']['date']
    
    if 'specifier' in raw_data[property_raw]:
        new_dic['Specifier'] = raw_data[property_raw]['specifier']
    else: 
        new_dic['Specifier'] = 'None'
    new_dic['Correctness'] = 'None'
    new_dic['Tag'] = 'CDE'
    
    new_dic['Info'] = {}
    if propertyname == 'Capacity':
        if 'current_value' in raw_data['BatteryCapacity']:
            new_dic['Info']['current_value'] = raw_data['BatteryCapacity']['current_value']
        if 'current_units' in raw_data['BatteryCapacity']:
            new_dic['Info']['current_units'] = raw_data['BatteryCapacity']['current_units']
        if 'cycle_value' in raw_data['BatteryCapacity']:
            new_dic['Info']['cycle_value'] = raw_data['BatteryCapacity']['cycle_value']
        if 'cycle_units' in raw_data['BatteryCapacity']:
            new_dic['Info']['cycle_units'] = raw_data['BatteryCapacity']['cycle_units']
    if new_dic['Info'] == {}:
        new_dic['Info'] = "None"
    if propertyname == 'Coulombic':
        new_dic['Property'] = 'Coulombic Efficiency'
    if 'warning' in list(raw_data.keys()):
        new_dic['Warning'] = 'R'
    else:
        new_dic['Warning'] = 'None'
            
    return new_dic

In [4]:
# Create the data tables as dictionaries and pandas dataframe.
newdata = []
for data in raw_data:
        newdata0 = convert_rawdata(data)
        newdata.append(newdata0)
df = pd.DataFrame(newdata)
df = df.drop(columns=['Value'])
df = df.loc[df.astype(str).drop_duplicates().index]

In [5]:
df

,Property,name,Raw_value,Raw_unit,Unit,DOI,Title,Journal,Date,Specifier,Correctness,Tag,Info,Warning
0,Voltage,[LiFePO4],3.7,V,Volt^(1.0),10.1039/A9RP90001B,TEACHING OF CHEMISTRY - LOGICAL OR PSYCHOLOGIC...,Chemistry Education Research and Practice,2000/01/01,voltage,None,CDE,None,R
1,Capacity,[graphite],372,mAh/g,Gram^(-1.0) Hour^(1.0) MilliAmpere^(1.0),10.1039/A9RP90001B,TEACHING OF CHEMISTRY - LOGICAL OR PSYCHOLOGIC...,Chemistry Education Research and Practice,2000/01/01,capacity,None,CDE,None,R


In [ ]:
def capfirst(s):
    """Capitalize the first letter. Remain unchanged for others."""
    return s[:1].upper() + s[1:]

In [ ]:
import re
def extract_value(string):
    """
    Takes a string and returns a list of floats representing the string given.
    Usage::
        test_string = '150 to 160'
        end_value = extract_value(test_string)
        print(end_value) # [150., 160.]
    :param str string: A representation of the values as a string
    :returns: The value expressed as a list of floats of length 1 if the value had no range,
        and as a list of floats of length 2 if it was a range.
    :rtype: list(float)
    """
    if string is None:
        return None
    string = string.replace("-", "-")
    string = string.replace("–", "-")
    string = string.replace("−", "-")
    string = string.replace(",", " ")
    string = string.split("±")[0]
    split_by_space = [r for r in re.split(' |(-)', string) if r]
    split_by_num = []
    for elem in split_by_space:
        split_by_num.extend([r for r in re.split('(\d+\.?(?:\d+)?)', elem) if r])
    try:
        if split_by_num[0] == "-":
            split_by_num[0] = "-" + split_by_num.pop(1)
    except:
        pass
    flag = 0
    new_split_by_num = []
    for index, value in enumerate(split_by_num):
        if flag == 2:
            new_split_by_num.append(split_by_num[index - 2])
            new_split_by_num.append(split_by_num[index - 1] + value)
            flag = 0
        elif flag == 1 and re.match('(-?\d+\.?(?:\d+)?)', value):
            new_split_by_num.append(split_by_num[index - 1])
            new_split_by_num.append(value)
            flag = 0
        elif not re.match('(-?\d+\.?(?:\d+)?)', value):
            flag += 1
        else:
            new_split_by_num.append(value)
    values = []
    for index, value in enumerate(new_split_by_num):
        try:
            float_val = float(value)
            float_val = abs(float_val)
            values.append(float_val)
        except ValueError:
            pass
    return values


def extract_conduct_number(number):
    """Extract conductivity value with power"""
    if re.findall(r'(\d+.)?\d+ × 10( )?[−−−-]( )?\d', number):
        numbers = extract_value(number)
        try:
            extracted_value = numbers[-3] * numbers[-2] ** -numbers[-1]
        except BaseException:
            return False
    elif re.findall(r'10( )?[−−−-]( )?\d', number):
        numbers = extract_value(number)
        extracted_value = numbers[-2] ** -numbers[-1]
    elif re.findall(r'(\d+.)?\d+ × 10\d', number):
        numbers = extract_value(number)
        try:
            extracted_value = numbers[-2] * 10 ** (numbers[-1] - 100)
        except BaseException:
            return False
    elif re.findall('^10[1-9]$', number):
        numbers = extract_value(number)
        extracted_value = 10 ** (numbers[-1] - 100)
    else:
        extracted_value = extract_value(number)
    return extracted_value


def extract_conduct_value(string):
    """
    Extract conductivity values
    :param string:
    :return:
    """
#     if re.match('[−−−-]? \d (and|to|[−−−–-]) \d', 'string'):
#         string_list = re.split('and|to|[^10][−−−–-]]', string)
        
    if 'and' in string or 'to' in string or "," in string:
        string_list = re.split('and|to|,', string)
    else:
        string_list = [string]
    extracted_values = []
    for number in string_list:
        extracted_value = extract_conduct_number(number)
        extracted_values.append(extracted_value)

    extracted_values_final = []
    for value in extracted_values:
        if type(value) != list:
            extracted_values_final.append(value)
        else:
            for value_in_list in value:
                if value_in_list != []:
                    extracted_values_final.append(value_in_list)      
    return extracted_values_final


In [ ]:
# Convert raw values into standard formula
value = df.apply(lambda df: extract_value(df['Raw_value']) if df['Property'] != "Conductivity" else extract_conduct_value(df['Raw_value']) , axis=1)

In [ ]:
# New dictionary df_dic with normalized value
df.insert(loc=4, column="value", value=value, allow_duplicates=False)
df.insert(loc=5, column='Value', value='v')
df.insert(loc=6, column='Name', value='n')
df_dic = df.to_dict(orient='records')

In [ ]:
import copy
df_dics = []
for i, row in enumerate(df_dic):
    name_list = [capfirst(na) for na in row['name']]
    name = list(sorted(set(name_list), key=name_list.index))
    value = row['value']
    row['Type'] = 'None'
    if len(name) == 1 and len(value) == 1:
        row['Name'] = capfirst(name[0])
        row['Value'] = value[0]
        df_dics.append(row)
    elif len(name) ==1 and len(value) >1:
        for v in value:
            new_row = copy.deepcopy(row)
            new_row['Name'] = capfirst(name[0]) 
            new_row['Value'] = v
            df_dics.append(new_row)
    elif len(name) >1 and len(value) == 1:
        for n in name:
            new_row = copy.deepcopy(row)
            new_row['Name'] = capfirst(n) 
            new_row['Value'] = value[0]
            df_dics.append(new_row)
    elif len(name) == len(value) and len(name)>1:
        for index in range(len(name)):
            new_row = copy.deepcopy(row)
            new_row['Name'] = capfirst(name[index])
            new_row['Value'] = value[index]
            df_dics.append(new_row)
    else:
        for nindex in range(len(name)):
            for vindex in range(len(value)):
                new_row = copy.deepcopy(row)
                new_row['Name'] = capfirst(name[nindex]) 
                new_row['Value'] = value[vindex]
                df_dics.append(new_row)

In [ ]:
from parse.material_parser import MaterialParser
def name_normalise(chems):
    """
    Normalize the chemical name as a dictionary of element names and its numbers. e.g. NaCl as {'Na':1, 'Cl':1}
    """
    mp = MaterialParser(pubchem_lookup=False, verbose=False)
    patter = r'He|Li|Be|Ne|Na|Mg|Al|Si|Cl|Ar|Ca|Sc|Ti|Cr|Mn|Fe|Co|Ni|Cu|Zn|Ga|Ge|As|Se|Br|Kr|Rb|Sr|Zr|Nb|Mo|Tc|Ru|' \
             r'Rh|Pd|Ag|Cd|In|Sn|Sb|Te|Xe|Cs|Ba|La|Ce|Pr|Nd|Pm|Sm|Eu|Gd|Tb|Dy|Ho|Er|Tm|Yb|Lu|Hf|Ta|Re|Os|Ir|Pt|Au|Hg' \
             r'|Tl|Pb|Bi|Po|At|Rn|Fr|Ra|Ac|Th|Pa|Np|Pu|Am|Cm|Bk|Cf|Es|Fm|Md|No|Lr|Rf|Db|Sg|Bh|Hs|Mt|Ds|Rg|Cn|Nh|Fl' \
             r'|Mc|Lv|Ts|Og|C|H|B|N|O|F|P|S|K|V|Y|I|W|U',
    pattern = patter[0].split('|')

    chem = chems.replace(' / ', '/').replace(' @ ', '@')
    if re.compile(r'\d/\d').search(chem) is None and '/' in chem:
        chem_list = chem.split('/')
    elif re.compile(r'\d@\d').search(chem) is None and '@' in chem:
        chem_list = chem.split('@')
    else:
        chem_list = [chem]
    normalized_name = []
    for cem in chem_list:
        try:
            parse = mp.parse_material(cem)
            if parse['composition'] != []:
                if parse['composition'][0]['elements'] != {}:
                    if all(
                            i in pattern for i in parse['composition'][0]['elements']):
                        normalized_name.append(
                            dict(parse['composition'][0]['elements']))
        except BaseException:
            continue
    return normalized_name, chems

In [ ]:
# Remove the reference electrode compound
from itertools import chain
reference_list=['Hg / Hg2Cl2', 'H2O / H2' ,'Ag / Ag2SO4','Pt / C','Hg / Hg2SO4','HgO / Hg','Hg / HgCl','AgCl / Ag','Hg / HgO','Ag / AgCl','acid','+']
reference_list0 = [i + 'electrode' for i in reference_list]
reference_list1 = [i + 'electrodes' for i in reference_list]
reference = reference_list + reference_list0 + reference_list1
chainrange0 = chain(range(40,42), range(48, 58),range(65,91),range(97,123))
chainrange = [i for i in chainrange0]

In [ ]:
# Extract the types for each data record, including Anode, Cathode, and Electrolyte.
new_dics = []
for i,row in enumerate(df_dics):
    print(row)
    name = row['Name']
    name = name.replace("composites","").replace("materials", "").replace("electrodes", "").replace("electrode", "").replace("material", "").replace("composite","")
    ordletter = row['Name'][0]
    ordletter1 = row['Name'][-1]
    if len(name) <=3 or name.endswith('/'):
        continue
    if name in reference or ord(ordletter) not in chainrange or ord(ordletter1) not in chainrange:
        continue
    if name.endswith('anode') or name.endswith('anodes'):
        name = " ".join(name.split()[0:-1])
        row['Type'] = 'Anode'
        print(row)
    elif name.endswith('cathode') or name.endswith('cathodes'):
        name = " ".join(name.split()[0:-1])
        row['Type'] = 'Cathode'
        print(row)
    elif name.endswith('electrolyte') or name.endswith('electrolytes'):
        name = " ".join(name.split()[0:-1])
        row['Type'] = 'Electrolyte'
        print(row)
    normalized_name,_ = name_normalise(name)
    if normalized_name:
        row['Name'] = name
        row['Extracted_name'] = normalized_name
        print(row)
        new_dics.append(row)

In [ ]:
new_dics

In [ ]:
# Standardlise different values and units as one formal unit
new_dics1=[]
for dat in new_dics:
    extract_name = dat['Extracted_name']
    if len(extract_name)==1 and len(extract_name[0])==1:
        continue
    if dat['Property']=='Voltage':
        if dat['Unit'] == 'MilliVolt^(1.0)':
            dat['Value'] = dat['Value']/1000
            dat['Unit'] = 'Volt^(1.0)'
        if dat['Unit'] == 'DimensionlessUnit':
            dat['Unit'] = 'Volt^(1.0)'
        if dat['Unit'] != 'Volt^(1.0)':
            continue
            
        if dat['Value'] > 5 or dat['Value'] < 1:
            if dat['Warning'] == 'R':
                dat['Warning'] = 'RL'
            else:
                dat['Warning'] = 'L'
        if dat['Value'] < 7 and dat['Value'] > 0.01:
            new_dics1.append(dat)
            
    if dat['Property'] == 'Capacity':
        if dat['Unit'] == 'Ampere^(1.0) Hour^(1.0) KiloGram^(-1.0)':
            dat['Unit'] = 'Gram^(-1.0)  Hour^(1.0)  MilliAmpere^(1.0)'
        if dat['Unit'] ==  'Ampere^(1.0)  Gram^(-1.0)  Hour^(1.0)':
            dat['Value'] = dat['Value']*1000
            dat['Unit'] = 'Gram^(-1.0)  Hour^(1.0)  MilliAmpere^(1.0)'
        if dat['Unit'] == 'Hour^(1.0)  KiloGram^(-1.0)  MilliAmpere^(1.0)':
            dat['Value'] = dat['Value']/1000
            dat['Unit'] = 'Gram^(-1.0)  Hour^(1.0)  MilliAmpere^(1.0)'
            
        if dat['Value'] > 3000 or dat['Value'] < 20:
            if dat['Warning'] == 'R':
                dat['Warning'] = 'RL'
            else:
                dat['Warning'] = 'L'
        if dat['Value'] < 5000 and dat['Value']>0:
            new_dics1.append(dat)
            
    if dat['Property'] == 'Conductivity':
        if dat['Unit'] == 'CentiMeter^(-1.0)  MilliSiemens^(1.0)':
            dat['Value'] = dat['Value']/1000
            dat['Unit'] = 'CentiMeter^(-1.0)  Siemens^(1.0)'
        if dat['Unit'] == 'Meter^(-1.0)  MilliSiemens^(1.0)':
            dat['Value'] = dat['Value']/100000
            dat['Unit'] = 'CentiMeter^(-1.0)  Siemens^(1.0)'
        if dat['Unit'] != 'CentiMeter^(-1.0)  Siemens^(1.0)':
            continue
            
        if dat['Value'] > 10:
            if dat['Warning'] == 'R':
                dat['Warning'] = 'RL'
            else:
                dat['Warning'] = 'L'
        if dat['Value'] < 100.1:
            new_dics1.append(dat)
        
    if dat['Property'] == 'Coulombic Efficiency':
        if dat['Value'] < 20:
            if dat['Warning'] == 'R':
                dat['Warning'] = 'RL'
            else:
                dat['Warning'] = 'L'
        if dat['Value'] <=100 and dat['Value']>0:
            new_dics1.append(dat)
    if dat['Property'] == 'Energy':
        if dat['Unit'] == 'Gram^(-1.0)  WattHour^(1.0)':
            dat['Value'] = dat['Value']*1000
            dat['Unit'] = 'KiloGram^(-1.0)  WattHour^(1.0)'
        if dat['Unit'] ==  'KiloGram^(1.0)  WattHour^(1.0)':
            dat['Unit'] ='KiloGram^(-1.0)  WattHour^(1.0)'
        if dat['Unit'] != 'KiloGram^(-1.0)  WattHour^(1.0)':
            continue
            
        if dat['Value'] > 3000 or dat['Value'] < 100:
            if dat['Warning'] == 'R':
                dat['Warning'] = 'RL'
            else:
                dat['Warning'] = 'L'
        if dat['Value'] <5000 and dat['Value']>0:
            new_dics1.append(dat)

In [ ]:
# Convert python dictionaries back into pandas dataframe again
df1 = pd.DataFrame(new_dics1)

cols_to_order = ['Property', 'Name', 'Value','Raw_unit','Raw_value','Unit','Extracted_name','DOI','Specifier','Tag','Warning','Type','Info']
new_columns = cols_to_order + (df1.columns.drop(cols_to_order).tolist())
df1 = df1[new_columns]

df1 = df1.drop_duplicates(subset=['Property','Value','DOI','Name'],keep='last')

In [ ]:
# Data augmentation preparation
group = df1.groupby('Name')
df2 = group.apply(lambda x: x['Property'].unique())
aug_names = []
for name0,propertylist in df2.iteritems():
    if 'Capacity' in propertylist and 'Voltage' in propertylist and 'Energy' not in propertylist:
        aug_names.append(name0)

In [ ]:
# Data augmentation for unknown energy data
for i,aug_name in enumerate(aug_names):
    print(i)
    newdf = df1[df1['Name']==aug_name]
    volt = newdf[newdf['Property']=='Voltage'].Value.mean()
    capa = newdf[newdf['Property']=='Capacity'].Value.mean()
    ener = volt*capa
    add = newdf.iloc[0].copy()
    add['Property'] = 'Energy'
    add['Value'] = ener
    add['Raw_value'] = str(ener)
    add['Raw_unit'] = 'Wh/kg'
    add['Unit'] = 'KiloGram^(-1.0)  WattHour^(1.0)'
    add['Tag'] = 'Calculated'
    add['DOI'], add['Specifier'], add['Warning'], add['Info'], add['Type'], add['Title'], add['Journal'], add['Date'] = 'None','None','None','None','None','None','None','None'
    df1 = df1.append(add)

In [ ]:
df2 = df1[~df1['Name'].str.startswith('–')]
df2 = df2.sort_values(by=['Name'])
df2 = df2.drop_duplicates(subset=['Property','Value','DOI','Name'],keep='last')
df2 = df2.sample(frac=1).reset_index(drop=True)
df2['id']=df2.index

In [ ]:
# Merge duplicated data with same property, value, and units.
def join(doi):
    return ', '.join((doi))
def unchange(item):
    return [item.iloc[:]]
df_merge_series = df2.copy()
df_merge_series['Count'] = 'None'
groups_series = df_merge_series.groupby(['Property','Name','DOI']).agg({'Count':'count','id':unchange})

In [ ]:
series_lists=groups_series[groups_series['Count']>3]['id'].values.tolist()
series=[]
for i in series_lists:
    new_list = i[0].values.tolist()
    for j in new_list:
        series.append(j)

In [ ]:
# Add series warning tag "S"
for i in series:
    if df2.iloc[i]['Warning']=="None":
        df2.set_value(i,'Warning','S')
    else:
        warning = df2.iloc[i]['Warning']
        df2.set_value(i,'Warning',warning+'S')

In [ ]:
# Other multiple warnings
df2=df2[df2['Warning']!="RLS"]
df2=df2[df2['Warning']!="RS"]
df2 = df2.drop(columns=['id'])
dfdic = df2.to_dict(orient='records')

In [ ]:
# Data merge
def join(doi):
    return ', '.join(doi)
def join_select(items):
    string = ''
    for i in items:
        if i != "None":
            string += str(i)
            string += ', '
    return string
def unchange(item):
    return item.iloc[0]
df_merge = df2.copy()
df_merge['Num_records'] = 'n'
# df.groupby(['Property','Value','Name'])['DOI'].apply(', '.join).reset_index()
groups = df_merge.groupby(['Property','Name','Value']).agg({'Raw_unit':unchange,'Raw_value':unchange,'Unit':unchange,'Num_records':'count',"Extracted_name":unchange,'DOI': join, "Specifier":join_select,  "Tag":unchange, "Warning":join_select,"Type":join_select,"Info":join_select,'Title': join, 'Journal':join, "Date":join,"Correctness":unchange})

merge = groups.reset_index()
merge = merge.drop_duplicates(subset=['Property','Value','DOI','Name'],keep='last')
df5 = merge.sample(frac=1).reset_index(drop=True)
merge = df5.replace([""],"None")
mergedic = merge.to_dict(orient='records')

#### Save the data records as json, csv and sqlite format

In [ ]:
import json
with open('save/battery.json', 'w') as fout:
    json.dump(dfdic, fout)

In [ ]:
import csv
csv_columns = list(df2.columns.values)
try:
    with open('battery.csv', 'a', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dfdic:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [ ]:
import sqlite3
inps = []
for i in dfdic:
    tuple1 = tuple(str(i[k]) for k in i)
    inps.append(tuple1)
conn = sqlite3.connect('battery' + '.db')

conn.execute('''CREATE TABLE BATTERY
            (Property       TEXT,
             Name           TEXT,
             Value          TEXT,
             Raw_unit       TEXT,
             Raw_value      TEXT,
             Unit           TEXT,
             Extracted_name TEXT,
             DOI            TEXT,
             Specifier      TEXT,
             Tag            TEXT,
             Warning       TEXT,
             Type          TEXT,
             Info          TEXT,
             Title         TEXT,
             Journal       TEXT,
             Date            TEXT,
             Correctness   TEXT)''')

db = conn.cursor()
db.executemany("INSERT INTO BATTERY VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", inps)
conn.commit()
conn.close()

Save the merged data records as json, csv and sqlite format

In [ ]:
import json
with open('battery_merged.json', 'w') as f:
    json.dump(mergedic, f)

In [ ]:
import csv
csv_columns = list(merge.columns.values)
try:
    with open('battery_merged.csv', 'a', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in mergedic:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [ ]:
import sqlite3
inps = []
for i in mergedic:
    tuple1 = tuple(str(i[k]) for k in i)
    inps.append(tuple1)
conn = sqlite3.connect('battery_merged' + '.db')

conn.execute('''CREATE TABLE BATTERY
            (Property       TEXT,
             Name           TEXT,
             Value          TEXT,
             Raw_unit       TEXT,
             Raw_value      TEXT,
             Unit           TEXT,
             Num_records    REAL,
             Extracted_name TEXT,
             DOI            TEXT,
             Specifier      TEXT,
             Tag            TEXT,
             Warning       TEXT,
             Type          TEXT,
             Info          TEXT,
             Title         TEXT,
             Journal       TEXT,
             Date            TEXT,
             Correctness   TEXT)''')

db = conn.cursor()
db.executemany("INSERT INTO BATTERY VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", inps)
conn.commit()
conn.close()